In [1]:
# To adapt Qt4 code: QtGui -> QtWidgets
from PyQt5 import QtWidgets, QtCore

from itertools import product, combinations

In [2]:
class char_placeholder():
    def __init__(self, name = 'None', element = 'None'):
        self.name = name
        self.element = element
        
class widget_char_placeholder(QtWidgets.QWidget):
    def __init__(self, char_type = char_placeholder):
        super(widget_char_placeholder, self).__init__()
        self.character = char_type()
        
def make_combo_box(caller, options, default_t, c_func = None):
    box = QtWidgets.QComboBox(caller)
    box.addItems(options)
    box.setCurrentText(default_t)
    if c_func: box.activated.connect(c_func)
    return box
        
def make_spin_box(caller, min_v, max_v, default_v, c_func = None):
    box = QtWidgets.QSpinBox(caller)
    box.setMinimum(min_v)
    box.setMaximum(max_v)
    box.setValue(default_v)
    if c_func: box.valueChanged.connect(c_func)
    return box

def make_double_spin_box(caller, min_v, max_v, default_v, c_func = None):
    box = QtWidgets.QDoubleSpinBox(caller)
    box.setMinimum(min_v)
    box.setMaximum(max_v)
    box.setValue(default_v)
    if c_func: box.valueChanged.connect(c_func)
    return box

def make_check_box(caller, text, default_s, c_func = None):
    box = QtWidgets.QCheckBox(text, caller)
    box.setChecked(default_s)
    if c_func: box.stateChanged.connect(c_func)
    return box

def make_basic_frame(layout):
    frame = QtWidgets.QFrame()
    frame.setLayout(layout)
    frame.setFrameStyle(QtWidgets.QFrame.StyledPanel)
    frame.setLineWidth(3)
    return frame

In [3]:
class widget_weapon(QtWidgets.QWidget):
    def __init__(self, holder = None, name = 'Basic weapon', atk_values = (380, 440, 500), asc_stat = 'None', asc_values = (0, 0, 0), psv_type = 'None', psv_range = (False,), psv_value = None):
        super(widget_weapon, self).__init__()
        # Because a weapon holds relatively little information, I did not create a non-widget class for them
        # Values are given for levels 70/70, 80/80, 90/90 because otherwise it's too much numbers and likely irrelevant anyhow.
        self.holder = holder # Required to pass some base stats for weapon passives
        self.name = name
        self.level = '90/90'
        self.refine = 1
        self.atk = {'70/70': atk_values[0], '80/80': atk_values[1], '90/90': atk_values[2]}
        self.asc = {'stat': asc_stat, '70/70': asc_values[0], '80/80': asc_values[1], '90/90': asc_values[2]}
        self.psv = {'type': psv_type, 'active': psv_range[0], 'value': psv_value} # Since this is in the widget itself, we don't need to keep track of psv_range itself
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_1 = QtWidgets.QHBoxLayout()
        label_level = QtWidgets.QLabel('Level:', self)
        self.box_level = make_combo_box(self, ('70/70', '80/80', '90/90'), self.level, self.update_weapon)
        label_refine = QtWidgets.QLabel('Refinement:', self)
        self.box_refine = make_spin_box(self, 1, 5, self.refine, self.update_weapon)
        label_atk = QtWidgets.QLabel('ATK:', self)
        self.widget_atk = QtWidgets.QLabel(str(self.atk[self.level]), self)
        self.widget_atk.setStyleSheet("font-weight: bold")
        label_asc = QtWidgets.QLabel('Ascension:', self)
        widget_asc_stat = QtWidgets.QLabel(self.asc['stat'], self) 
        widget_asc_stat.setStyleSheet("font-weight: bold")
        self.widget_asc_value = QtWidgets.QLabel(('' if self.asc['stat'] == 'None' else str(self.asc[self.level]) + ('%' if self.asc['stat'] not in ('EM', 'CV') else '')), self) 
        self.widget_asc_value.setStyleSheet("font-weight: bold")
        for item in (label_level, self.box_level, label_refine, self.box_refine, label_atk, self.widget_atk, label_asc, widget_asc_stat, self.widget_asc_value): layout_1.addWidget(item)
        layout_1.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        layout_2 = QtWidgets.QHBoxLayout()
        label_psv = QtWidgets.QLabel('Passive:', self)
        widgets_to_add = [label_psv]
        if self.psv['type'] == 'Bool':
            self.widget_psv = make_check_box(self, '', self.psv['active'], self.update_weapon)
            widgets_to_add = widgets_to_add + [self.widget_psv]
        elif self.psv['type'] == 'Stacks':
            self.widget_psv = make_spin_box(self, psv_range[0], psv_range[1], self.psv['active'], self.update_weapon)
            label_passive_extra = QtWidgets.QLabel('stacks', self)
            widgets_to_add = widgets_to_add + [self.widget_psv, label_passive_extra]
        elif self.psv['type'] == 'Perc':
            self.widget_psv = make_double_spin_box(self, psv_range[0], psv_range[1], self.psv['active'], self.update_weapon)
            label_passive_extra = QtWidgets.QLabel('Perc', self)
            widgets_to_add = widgets_to_add + [self.widget_psv, label_passive_extra]
        else:
            label_passive_extra = QtWidgets.QLabel(self.psv['type'], self)
            widgets_to_add = widgets_to_add + [label_passive_extra]
        for item in widgets_to_add: layout_2.addWidget(item)
        layout_2.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        layout_main.addLayout(layout_1)
        layout_main.addLayout(layout_2)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    def update_weapon(self):
        self.level = self.box_level.currentText()
        self.refine = self.box_refine.value()
        self.widget_atk.setText(str(self.atk[self.level]))
        self.widget_asc_value.setText(('' if self.asc['stat'] == 'None' else str(self.asc[self.level]) + ('%' if self.asc['stat'] not in ('EM', 'CV') else '')))
        if self.psv['type'] == 'Bool':
            self.psv['active'] = self.widget_psv.isChecked()
        elif self.psv['type'] == 'Stacks':
            self.psv['active'] = self.widget_psv.value()
        elif self.psv['type'] == 'Perc':
            self.psv['active'] = self.widget_psv.value()
        else:
            return            
            
    def apply_buffs(self, stats):
        # Apply weapon buffs to stats: basic weapon has none
        return stats

    def apply_conversions(self, stats):
        # Apply conversions to stats: basic weapon has none
        return stats
        
class char_hypercarry(char_placeholder):
    def __init__(self, name = 'Basic hypercarry', element = 'Physical', scales_with = ('ATK', 'DMG', 'CV')):
        super(char_hypercarry, self).__init__(name, element)
        self.scales_with = scales_with # What stats do we need to show on the widget? Full list of options is  ('HP', 'ATK', 'DEF', 'EM', 'ER', 'DMG', 'CV')
        # Base stats: character ascension
        self.base_stats = {
            'HP': 10000,
            'ATK': 300,
            'DEF': 800,
            'asc_stat': 'None', # Base hypercarry does not have a relevant ascension stat
            'asc_value': 0
        }
        # Weapons
        self.weapon_types = {
            'Abstract weapon': widget_weapon_custom(self)
        }
        self.weapon = self.weapon_types['Abstract weapon'] # Since this is a reference and not a copy, it can be direct like this instead of an index with same effect
        # Extra stats - artifact substats and non-accounted-for effects
        self.extra_stats = {
            'HP_flat': 4780, # From sources such as the flower or rare weapon effects
            'HP_perc': 0,
            'ATK_flat': 311,
            'ATK_perc': 0,
            'DEF_flat': 0,
            'DEF_perc': 0,
            'EM': 0,
            'ER': 0,
            'DMG': 0, # Ignoring the goblet mainstat, but including such things as artifact 2p effects
            'CV': 170 # Base level invested
        }
        # # Artefact types that interest us for this hypercarry
        self.sands_types = (['Other'] 
                            + (['HP%'] if 'HP' in self.scales_with else [])
                            + (['ATK%'] if 'ATK' in self.scales_with else [])
                            + (['DEF%'] if 'DEF' in self.scales_with else [])
                            + (['EM'] if 'EM' in self.scales_with else [])
                            + (['ER%'] if 'ER' in self.scales_with else []))
        self.goblet_types = (['Other']
                            + (['HP%'] if 'HP' in self.scales_with else [])
                            + (['ATK%'] if 'ATK' in self.scales_with else [])
                            + (['DEF%'] if 'DEF' in self.scales_with else [])
                            + (['EM'] if 'EM' in self.scales_with else [])
                            + (['Elemental DMG%'] if 'DMG' in self.scales_with else []))
        self.circlet_types = (['Other']
                            + (['HP%'] if 'HP' in self.scales_with else [])
                            + (['ATK%'] if 'ATK' in self.scales_with else [])
                            + (['DEF%'] if 'DEF' in self.scales_with else [])
                            + (['EM'] if 'EM' in self.scales_with else [])
                            + (['CRIT'] if 'CV' in self.scales_with else []))
        self.artifacts = ['Other', 'Other', 'Other']
        
    def base_atk(self, weapon = None):
        if weapon == None:
            weapon = self.weapon
        return self.base_stats['ATK'] + weapon.atk[weapon.level]
    # Note: this one function - or rather the requirement of the 'hypercarry base attack' value for both character and weapon buff calculations complicates things immensely
    # Either a weapon has to be an object referenced as property of a character - in which case it has to contain holder reference which isn't good practice,
    # or it has to be contained within the character object itself - in which case we can't override weapons in a convenient manner
    # Solution: rewrite it so that 'hypercarry widget' contains a character widget, a weapon widget, and calls buff functions itself, passing one to another when necessary
        
    def calculate_stats(self, weapon = None, artifacts = None):
        # Calculating 'internal' stats. Allows me to not have to change the calculate_output method for specific hypercarries
        stats = {}
        if weapon == None:
            weapon = self.weapon
        if artifacts == None: # No build override
            sands = self.artifacts[0]
            goblet = self.artifacts[1]
            circlet = self.artifacts[2]
        else: # Overriding artifacts with provided
            sands = artifacts[0]
            goblet = artifacts[1]
            circlet = artifacts[2]
        stats['HP'] = (1 
                       + (0.466 if sands == 'HP%' else 0) + (0.466 if goblet == 'HP%' else 0) + (0.466 if circlet == 'HP%' else 0) 
                       + self.extra_stats['HP_perc']/100)*self.base_stats['HP'] + self.extra_stats['HP_flat']
        stats['ATK'] = (1 
                        + (0.466 if sands == 'ATK%' else 0) + (0.466 if goblet == 'ATK%' else 0) + (0.466 if circlet == 'ATK%' else 0) 
                        + self.extra_stats['ATK_perc']/100)*self.base_atk(weapon) + self.extra_stats['ATK_flat']
        stats['DEF'] = (1 
                        + (0.583 if sands == 'DEF%' else 0) + (0.583 if goblet == 'DEF%' else 0) + (0.583 if circlet == 'DEF%' else 0) 
                        + self.extra_stats['DEF_perc']/100)*self.base_stats['DEF'] + self.extra_stats['DEF_flat']
        stats['EM'] = 0 + (186.5 if sands == 'EM' else 0) + (186.5 if goblet == 'EM' else 0) + (186.5 if circlet == 'EM' else 0) + self.extra_stats['EM']
        stats['ER'] = 100 + (51.8 if sands == 'ER%' else 0) + self.extra_stats['ER']
        stats['DMG'] = 0 + ((58.3 if self.element == 'Physical' else 46.6) if goblet == 'Elemental DMG%' else 0) + self.extra_stats['DMG']
        stats['CV'] = 5*2 + 50 + (62.2 if circlet == 'CRIT' else 0) + self.extra_stats['CV'] # 'Effective CV' as a plain sum of 2x CR and CD, assuming balanced stats
        # Ascension stats
        asc_stat_char = self.base_stats['asc_stat']
        asc_stat_weap = weapon.asc['stat']
        stat_mults = (self.base_stats['HP']/100, self.base_atk(weapon)/100, self.base_stats['DEF']/100, 1, 1, 1, 1)
        if asc_stat_char != 'None':
            stats[asc_stat_char] = stats[asc_stat_char] + self.base_stats['asc_value']*stat_mults[list(stats.keys()).index(asc_stat_char)]
        if asc_stat_weap != 'None':
            stats[asc_stat_weap] = stats[asc_stat_weap] + weapon.asc[weapon.level]*stat_mults[list(stats.keys()).index(asc_stat_weap)]
        return stats
    
    def apply_self_buffs(self, stats):
        # Independent self-buffs such as ascension passives giving flat stat increases that can be assumed to be always active
        return stats

    def apply_conversions(self, stats):
        # Post-buff conversions such as Hu Tao's PP and Itto's burst. Basic hypercarry doesn't have any self-buffs, so the values are just returned
        return stats

class widget_hypercarry(widget_char_placeholder):
    def __init__(self, char_type = char_hypercarry):
        super(widget_hypercarry, self).__init__(char_type)
        layout_main = QtWidgets.QVBoxLayout(self)
        # Base stats
        label_stats_base = QtWidgets.QLabel('Base stats:', self)
        layout_stats_base = QtWidgets.QHBoxLayout()
        label_element = QtWidgets.QLabel('Element:', self)
        self.widget_element = QtWidgets.QLabel(self.character.element, self) # A property so that subclasses can change it easily
        self.widget_element.setStyleSheet("font-weight: bold")
        widgets_to_add = [label_element, self.widget_element]
        if 'HP' in self.character.scales_with: # All these lines and similar in extra can be optimized further with a loop if I keep stats in a list, but they require different kinds of boxes too
            label_base_hp = QtWidgets.QLabel('HP:', self)
            self.widget_base_hp = QtWidgets.QLabel(str(self.character.base_stats['HP']), self)
            self.widget_base_hp.setStyleSheet("font-weight: bold")
            widgets_to_add = widgets_to_add + [label_base_hp, self.widget_base_hp]
        if 'ATK' in self.character.scales_with:
            label_base_atk = QtWidgets.QLabel('ATK:', self)
            self.widget_base_atk = QtWidgets.QLabel(str(self.character.base_stats['ATK']), self)
            self.widget_base_atk.setStyleSheet("font-weight: bold")
            widgets_to_add = widgets_to_add + [label_base_atk, self.widget_base_atk]
        if 'DEF' in self.character.scales_with:
            label_base_def = QtWidgets.QLabel('DEF:', self)
            self.widget_base_def = QtWidgets.QLabel(str(self.character.base_stats['DEF']), self)
            self.widget_base_def.setStyleSheet("font-weight: bold")
            widgets_to_add = widgets_to_add + [label_base_def, self.widget_base_def]
        label_ascension = QtWidgets.QLabel('Ascension stat:', self)
        self.widget_asc_stat = QtWidgets.QLabel(self.character.base_stats['asc_stat'], self)
        self.widget_asc_stat.setStyleSheet("font-weight: bold")
        self.widget_asc_value = QtWidgets.QLabel(('' if self.character.base_stats['asc_stat'] == 'None'
                                                  else str(self.character.base_stats['asc_value']) + ('%' if self.character.base_stats['asc_stat'] != 'CV' else '')), self)
        self.widget_asc_value.setStyleSheet("font-weight: bold")
        widgets_to_add = widgets_to_add + [label_ascension, self.widget_asc_stat, self.widget_asc_value]
        for item in widgets_to_add: layout_stats_base.addWidget(item)
        layout_stats_base.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        # Weapon
        layout_label_weapon = QtWidgets.QHBoxLayout()
        label_weapon = QtWidgets.QLabel('Weapon:', self)
        self.box_weapon = make_combo_box(self, tuple(self.character.weapon_types.keys()), self.character.weapon.name, self.update_weapon)
        layout_label_weapon.addWidget(label_weapon)
        layout_label_weapon.addWidget(self.box_weapon)
        layout_label_weapon.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout_weapon = QtWidgets.QStackedLayout()
        for item in self.character.weapon_types.values(): self.layout_weapon.addWidget(item)
        self.layout_weapon.setCurrentIndex(tuple(self.character.weapon_types.keys()).index(self.character.weapon.name))
        frame_weapon = make_basic_frame(self.layout_weapon)
        # Artifacts
        label_artifacts = QtWidgets.QLabel('Artifacts: types by main stat', self)
        layout_artifacts = QtWidgets.QHBoxLayout()
        label_sands = QtWidgets.QLabel('Sands:', self)
        self.box_sands = make_combo_box(self, self.character.sands_types, self.character.artifacts[0], self.update_char)
        label_goblet = QtWidgets.QLabel('Goblet:', self)
        self.box_goblet = make_combo_box(self, self.character.goblet_types, self.character.artifacts[1], self.update_char)
        label_circlet = QtWidgets.QLabel('Circlet:', self)
        self.box_circlet = make_combo_box(self, self.character.circlet_types, self.character.artifacts[2], self.update_char)
        layout_artifacts = QtWidgets.QHBoxLayout(self)
        for item in (label_sands, self.box_sands, label_goblet, self.box_goblet, label_circlet, self.box_circlet): layout_artifacts.addWidget(item)
        layout_artifacts.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        # Extra stats
        label_stats_extra = QtWidgets.QLabel('Extra stats: artifact substats, set bonuses', self)
        layout_stats_extra = QtWidgets.QHBoxLayout()
        widgets_to_add = []
        self.boxes_extra = {}
        for stat in self.character.scales_with:
            if stat in ('HP', 'ATK', 'DEF'):
                layout_stat = QtWidgets.QGridLayout()
                label_stat_flat = QtWidgets.QLabel('Flat ' + stat + ':', self)
                self.boxes_extra[stat + '_flat'] = make_spin_box(self, 0, 9999, self.character.extra_stats[stat + '_flat'], self.update_char)
                label_stat_perc = QtWidgets.QLabel(stat + '%:', self)
                self.boxes_extra[stat + '_perc'] = make_double_spin_box(self, 0, 999, self.character.extra_stats[stat + '_perc'], self.update_char)
                layout_stat.addWidget(label_stat_flat, 0, 0)
                layout_stat.addWidget(label_stat_perc, 1, 0)
                layout_stat.addWidget(self.boxes_extra[stat + '_flat'], 0, 1)
                layout_stat.addWidget(self.boxes_extra[stat + '_perc'], 1, 1)
                widgets_to_add = widgets_to_add + [layout_stat]
                del layout_stat
                del label_stat_flat
                del label_stat_perc
            elif stat == 'EM': # EM is an integer unlke all other single-number stats
                label_stat = QtWidgets.QLabel(stat + ':', self)
                self.boxes_extra[stat] = make_spin_box(self, 0, 999, self.character.extra_stats[stat], self.update_char)
                widgets_to_add = widgets_to_add + [label_stat, self.boxes_extra[stat]]
                del label_stat
            else:
                label_stat = QtWidgets.QLabel(stat + ':', self)
                self.boxes_extra[stat] = make_double_spin_box(self, 0, 999, self.character.extra_stats[stat], self.update_char)
                widgets_to_add = widgets_to_add + [label_stat, self.boxes_extra[stat]]
                del label_stat
        for item in widgets_to_add:
            if type(item) in (QtWidgets.QHBoxLayout, QtWidgets.QVBoxLayout, QtWidgets.QGridLayout, QtWidgets.QStackedLayout): layout_stats_extra.addLayout(item)
            else: layout_stats_extra.addWidget(item)
        layout_stats_extra.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        for item in (label_stats_base, layout_stats_base, layout_label_weapon, frame_weapon, label_artifacts, layout_artifacts, label_stats_extra, layout_stats_extra):
            if type(item) in (QtWidgets.QHBoxLayout, QtWidgets.QVBoxLayout, QtWidgets.QGridLayout, QtWidgets.QStackedLayout): layout_main.addLayout(item)
            else: layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))

    def update_weapon(self):
        weapon_name = self.box_weapon.currentText()
        if weapon_name != self.character.weapon.name:
            self.layout_weapon.setCurrentIndex(tuple(self.character.weapon_types.keys()).index(weapon_name))
            self.character.weapon = self.layout_weapon.currentWidget() # Theoretically, any calculation character does with the weapon can be handled by character object asking it's parent widget
        
    def update_char(self):
        for stat in self.character.scales_with:
            if stat in ('HP', 'ATK', 'DEF'):
                self.character.extra_stats[stat + '_flat'] = self.boxes_extra[stat + '_flat'].value()
                self.character.extra_stats[stat + '_perc'] = self.boxes_extra[stat + '_perc'].value()
            else:
                self.character.extra_stats[stat] = self.boxes_extra[stat].value()
        self.character.artifacts = [self.box_sands.currentText(), self.box_goblet.currentText(), self.box_circlet.currentText()] # Maybe this should be a tuple

In [4]:
class widget_weapon_custom(QtWidgets.QWidget):
    def __init__(self, holder = None):
        super(widget_weapon_custom, self).__init__()
        # Custom weapon: customizable stats, no passive
        self.name = 'Abstract weapon'
        self.level = 0
        self.atk = [500]
        self.asc = {'stat': 'None', 'value': 0}
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_1 = QtWidgets.QHBoxLayout()
        label_atk = QtWidgets.QLabel('ATK:', self)
        self.box_atk = make_spin_box(self, 1, 1000, self.atk[self.level], self.update_weapon)
        label_asc = QtWidgets.QLabel('Ascension:', self)
        self.box_asc_stat = make_combo_box(self, ['None'] + list(holder.scales_with), self.asc['stat'], self.update_weapon)
        self.box_asc_value = make_double_spin_box(self, 0, 1000, self.asc['value'], self.update_weapon)
        for item in (label_atk, self.box_atk, label_asc, self.box_asc_stat, self.box_asc_value): layout_1.addWidget(item)
        layout_1.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        layout_2 = QtWidgets.QHBoxLayout()
        label_psv = QtWidgets.QLabel('No passive supported for an absctract weapon', self)
        layout_2.addWidget(label_psv)
        layout_2.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        layout_main.addLayout(layout_1)
        layout_main.addLayout(layout_2)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    def update_weapon(self):
        self.atk[self.level] = self.box_atk.value()
        self.asc['stat'] = self.box_asc_stat.currentText()
        self.asc['value'] = self.box_asc_value.value()
            
    def apply_buffs(self, stats):
        # Apply weapon buffs to stats: basic weapon has none
        return stats

    def apply_conversions(self, stats):
        # Apply conversions to stats: basic weapon has none
        return stats

class char_hypercarry_custom(char_hypercarry):
    def __init__(self):
        super(char_hypercarry_custom, self).__init__('Abstract hypercarry', 'Physical', ('HP', 'ATK', 'DEF', 'EM', 'ER', 'DMG', 'CV'))
        
class widget_hypercarry_custom(widget_hypercarry):
    def __init__(self, char_type = char_hypercarry_custom):
        super(widget_hypercarry_custom, self).__init__(char_type)
        # Base stats: replace base stats with selectors
        self.box_element = make_combo_box(self, ('Physical', 'Anemo', 'Pyro', 'Hydro', 'Electro', 'Cryo', 'Geo', 'Dendro'), self.character.element, self.update_char_extras)
        self.box_base_hp = make_spin_box(self, 1, 50000, self.character.base_stats['HP'], self.update_char_extras)
        self.box_base_atk = make_spin_box(self, 1, 1000, self.character.base_stats['ATK'], self.update_char_extras)
        self.box_base_def = make_spin_box(self, 1, 2000, self.character.base_stats['DEF'], self.update_char_extras)
        self.box_asc_stat = make_combo_box(self, ['None'] + list(self.character.scales_with), self.character.base_stats['asc_stat'], self.update_char_extras)
        self.box_asc_value = make_double_spin_box(self, 0, 1000, self.character.base_stats['asc_value'], self.update_char_extras)
        for old_w, new_w in zip((self.widget_element, self.widget_base_hp, self.widget_base_atk, self.widget_base_def, self.widget_asc_stat, self.widget_asc_value),
                                (self.box_element, self.box_base_hp, self.box_base_atk, self.box_base_def, self.box_asc_stat, self.box_asc_value)):
            self.layout().itemAt(1).replaceWidget(old_w, new_w)
            old_w.hide()
        # Weapon: no changes
        # Artifacts: no changes
        # Extra stats: no changes
        # Character-specific extras: nothing

    def update_char_extras(self):
        self.character.element = self.box_element.currentText()
        if 'HP' in self.character.scales_with: self.character.base_stats['HP'] = self.box_base_hp.value()
        if 'ATK' in self.character.scales_with: self.character.base_stats['ATK'] = self.box_base_atk.value()
        if 'DEF' in self.character.scales_with: self.character.base_stats['DEF'] = self.box_base_def.value()
        self.character.base_stats['asc_stat'] = self.box_asc_stat.currentText()
        self.character.base_stats['asc_value'] = self.box_asc_value.value()

In [5]:
class char_ayaka(char_hypercarry):
    def __init__(self):
        super(char_ayaka, self).__init__('Kamisato Ayaka', 'Cryo', ('ATK', 'DMG', 'CV'))
        # Default Ayaka is assumed to be lvl 90, holding a Amenoma Kageuchi and 4p Blizzard Strayer. C4 is currently not counted because it is finicky
        self.base_stats['HP'] = 12858
        self.base_stats['ATK'] = 342
        self.base_stats['DEF'] = 783
        self.base_stats['asc_stat'] = 'CV'
        self.base_stats['asc_value'] = 38.4
        self.weapon_types.update({
            'Amenoma Kageuchi': widget_amenoma_kageuchi(self),
            'Finale of the Deep': widget_finale_of_the_deep(self),
            'Mistsplitter Reforged': widget_mistsplitter_reforged(self)
        })
        self.weapon = self.weapon_types['Amenoma Kageuchi']
        self.bs4p = '4p Freeze'

    def apply_self_buffs(self, stats):
        # Ayaka benefits from A1 (negligible so doesn't count) and A4, can benefit from 4p BS
        stats['DMG'] = stats['DMG'] + 18
        stats['CV'] = stats['CV'] + (40 if self.bs4p == '4p Cryo' else 0) + (80 if self.bs4p == '4p Freeze' else 0)
        return stats
    
class widget_ayaka(widget_hypercarry):
    def __init__(self):
        super(widget_ayaka, self).__init__(char_ayaka)
        layout_extras = QtWidgets.QHBoxLayout()
        label_bs4p = QtWidgets.QLabel('Blizzard Strayer:', self)
        self.box_bs4p = make_combo_box(self, ('No 4p bonus', '4p Cryo', '4p Freeze'), self.character.bs4p, self.update_char_extras)
        layout_extras.addWidget(label_bs4p)
        layout_extras.addWidget(self.box_bs4p)
        layout_extras.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout().insertLayout(self.layout().count() - 1, layout_extras)
    
    def update_char_extras(self):
        self.character.bs4p = self.box_bs4p.currentText()
        
class char_hutao(char_hypercarry):
    def __init__(self):
        super(char_hutao, self).__init__('Hu Tao', 'Pyro', ('HP', 'ATK', 'DMG', 'CV'))
        # Default Hu Tao is assumed to be lvl 90, holding a 500 ATK weapon and 4p Crimson Witch of Flames, healthy. Constellations are currently not counted because they are finicky
        self.base_stats['HP'] = 15552
        self.base_stats['ATK'] = 106
        self.base_stats['DEF'] = 876
        self.base_stats['asc_stat'] = 'HP'
        self.base_stats['asc_value'] = 38.4
        self.talent = 8
        self.under_half_hp = False
        self.a_set_types = ('No 4p set bonus', 'Crimson Witch of Flames', 'Shimenawa\'s Reminiscence')
        self.artifact_set = self.a_set_types[1]

    def apply_self_buffs(self, stats):
        # Hu Tao can benefit from A4, 4p CW and 4p SR
        stats['DMG'] = stats['DMG'] + (33 if self.under_half_hp else 0) + (0, 15*0.5, 50)[self.a_set_types.index(self.artifact_set)]
        return stats

    def apply_conversions(self, stats):
        # Hu Tao converts max HP to ATK
        stats['ATK'] = stats['ATK'] + min(self.base_atk()*4, stats['HP']*((3.84, 4.07, 4.3, 4.6, 4.83, 5.06, 5.36, 5.66, 5.96, 6.26, 6.55, 6.85, 7.15)[self.talent])/100)
        return stats
    
class widget_hutao(widget_hypercarry):
    def __init__(self):
        super(widget_hutao, self).__init__(char_hutao)
        layout_extras = QtWidgets.QHBoxLayout()
        label_talent = QtWidgets.QLabel('Talent level:', self)
        self.box_talent = make_spin_box(self, 1, 13, self.character.talent, self.update_char_extras)
        self.box_under_half_hp = make_check_box(self, 'Is under 50% HP', self.character.under_half_hp, self.update_char_extras)
        label_artifact_set = QtWidgets.QLabel('Artifact set bonus:', self)
        self.box_artifact_set = make_combo_box(self, self.character.a_set_types, self.character.artifact_set, self.update_char_extras)
        for item in (label_talent, self.box_talent, self.box_under_half_hp, label_artifact_set, self.box_artifact_set): layout_extras.addWidget(item)
        layout_extras.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout().insertLayout(self.layout().count() - 1, layout_extras)
    
    def update_char_extras(self):
        self.character.talent = self.box_talent.value()
        self.character.under_half_hp = self.box_under_half_hp.isChecked()
        self.character.artifact_set = self.box_artifact_set.currentText()
        
class char_ei(char_hypercarry):
    def __init__(self):
        super(char_ei, self).__init__('Raiden Shogun', 'Electro', ('ATK', 'ER', 'DMG', 'CV'))
        # Default Ei is assumed to be lvl 90 holding The Catch R5 at lvl 90 and 4p Emblem of Severed Fate
        self.base_stats['HP'] = 12907
        self.base_stats['ATK'] = 337
        self.base_stats['DEF'] = 789
        self.base_stats['asc_stat'] = 'ER'
        self.base_stats['asc_value'] = 32
        self.weapon_types.update({
            'The Catch': widget_the_catch(self),
            'Engulfing Lightning': widget_engulfing_lightning(self)
        })
        self.weapon = self.weapon_types['The Catch']
        self.weapon_types['The Catch'].widget_psv.setChecked(True) # Since Ei uses burst in HC mode
        self.weapon_types['Engulfing Lightning'].widget_psv.setChecked(True)
        self.eosf4p = True
    
    def apply_conversions(self, stats):
        # Ei converts ER to DMG% with her A4, can benefit from 4p EosF
        stats['DMG'] = stats['DMG'] + 0.4*(stats['ER'] - 100) + (max(0.25*stats['ER'], 75) if self.eosf4p else 0)
        return stats

class widget_ei(widget_hypercarry):
    def __init__(self):
        super(widget_ei, self).__init__(char_ei)
        layout_extras = QtWidgets.QHBoxLayout()
        self.box_eosf4p = make_check_box(self, 'Holds 4p Emblem of Severed Fate', self.character.eosf4p, self.update_char_extras)
        layout_extras.addWidget(self.box_eosf4p)
        layout_extras.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout().insertLayout(self.layout().count() - 1, layout_extras)
    
    def update_char_extras(self):
        self.character.eosf4p = self.box_eosf4p.isChecked()
        
class char_diluc(char_hypercarry):
    def __init__(self):
        super(char_diluc, self).__init__('Diluc', 'Pyro', ('ATK', 'DMG', 'CV'))
        # Default Diluc is assumed to be lvl 90 holding a 500 ATK weapon and 4p Crimson Witch of Flames. Constellations are currently not counted because they are finicky
        self.base_stats['HP'] = 12980
        self.base_stats['ATK'] = 334
        self.base_stats['DEF'] = 783
        self.base_stats['asc_stat'] = 'CV'
        self.base_stats['asc_value'] = 38.4
        self.weapon_types.update({
            'Serpent Spine': widget_serpent_spine(self)
        })
        self.cwof4p = True
    
    def apply_self_buffs(self, stats):
        # Diluc benefits from A4, and can benefit from 4p CW
        stats['DMG'] = stats['DMG'] + 20 + (15*1.5 if self.cwof4p else 0)
        return stats

class widget_diluc(widget_hypercarry):
    def __init__(self):
        super(widget_diluc, self).__init__(char_diluc)
        layout_extras = QtWidgets.QHBoxLayout()
        self.box_cwof4p = make_check_box(self, 'Holds 4p Crimson Witch', self.character.cwof4p, self.update_char_extras)
        layout_extras.addWidget(self.box_cwof4p)
        layout_extras.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout().insertLayout(self.layout().count() - 1, layout_extras)
    
    def update_char_extras(self):
        self.character.cwof4p = self.box_cwof4p.isChecked()

In [6]:
class widget_amenoma_kageuchi(widget_weapon):
    def __init__(self, holder = None):
        super(widget_amenoma_kageuchi, self).__init__(holder, 'Amenoma Kageuchi', (347, 401, 454), 'ATK', (45.4, 50.3, 55.1), 'Does not affect stats directly')

class widget_finale_of_the_deep(widget_weapon):
    def __init__(self, holder = None):
        super(widget_finale_of_the_deep, self).__init__(holder, 'Finale of the Deep', (429, 497, 565), 'ATK', (22.7, 25.1, 27.6), 'Stacks', (0, 2), ((12, 150), (15, 187.5), (18, 225), (21, 262.5), (24, 300)))
        
    def apply_buffs(self, stats):
        # Finale of the Deep increases ATK by psv_value[0] after using a skill and further by psv_value[1] if the lifebond is cleared
        stats['ATK'] = stats['ATK'] + (self.psv['value'][self.refine - 1][0] if self.psv['active'] >= 1 else 0)*self.holder.base_atk(self)/100
        stats['ATK'] = stats['ATK'] + (self.psv['value'][self.refine - 1][1] if self.psv['active'] >= 2 else 0)
        return stats

class widget_mistsplitter_reforged(widget_weapon):
    def __init__(self, holder = None):
        super(widget_mistsplitter_reforged, self).__init__(holder, 'Mistsplitter Reforged', (506, 590, 674), 'CV', (36.3, 40.2, 44.1), 'Stacks', (0, 3), ((12, 8), (15, 10), (18, 12), (21, 14), (24, 16)))
        
    def apply_buffs(self, stats):
        # Mistsplitter Reforged provides an unconditional psv_value[0] DMG% bonus and 1x/2x/3.5x psv_value[1] DMG% bonus from stacks
        stats['DMG'] = stats['DMG'] + self.psv['value'][self.refine - 1][0] + self.psv['value'][self.refine - 1][1]*(0, 1, 2, 3.5)[self.psv['active']]
        return stats
    
class widget_the_catch(widget_weapon):
    def __init__(self, holder = None):
        super(widget_the_catch, self).__init__(holder, 'The Catch', (388, 449, 510), 'ER', (37.9, 41.9, 45.9), 'Bool', (False, True), ((16, 6), (20, 7.5), (24, 9), (28, 9), (32, 12)))
        # The Catch is assumed to be R5 because why not
        self.box_refine.setValue(5)
        
    def apply_buffs(self, stats):
        # The Catch increases DMG by psv_value[0] and CRIT Rate by psv_value[1] if what we are calculating for is an elemental burst
        stats['DMG'] = stats['DMG'] + (self.psv['value'][self.refine - 1][0] if self.psv['active'] == True else 0)
        stats['CV'] = stats['CV'] + (self.psv['value'][self.refine - 1][1]*2 if self.psv['active'] == True else 0)
        return stats
        
class widget_engulfing_lightning(widget_weapon):
    def __init__(self, holder = None):
        super(widget_engulfing_lightning, self).__init__(holder, 'Engulfing Lightning', (457, 532, 608), 'ER', (45.4, 50.3, 55.1), 'Bool', (False, True), ((28, 80), (35, 90), (42, 100), (49, 110), (56, 120)))

    def apply_buffs(self, stats):
        # Engulfing Lightning increases ER by 30% if what we are calculating for is an elemental burst
        stats['ER'] = stats['ER'] + (30 if self.psv['active'] == True else 0)
        return stats

    def apply_conversions(self, stats):
        # Engulfing Lightning converts ER to ATK
        stats['ATK'] = stats['ATK'] + max(self.psv['value'][self.refine - 1][1], self.psv['value'][self.refine - 1][0]*(stats['ER']/100 - 1))*self.holder.base_atk(self)/100
        return stats
    
class widget_serpent_spine(widget_weapon):
    def __init__(self, holder = None):
        super(widget_serpent_spine, self).__init__(holder, 'Serpent Spine', (388, 449, 510), 'CV', (45.4, 50.3, 55.1), 'Stacks', (0, 5), (6, 7, 8, 9, 10))
        
    def apply_buffs(self, stats):
        # Serpent Spine provides psv_value DMG% bonus for every stack
        stats['DMG'] = stats['DMG'] + self.psv['value'][self.refine - 1]*self.psv['active']
        return stats
    

In [7]:
# Relevancy list: buffers
# VV: "hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo')"
# Bennett: "'ATK' in hypercarry.scales_with or ('DMG' in hypercarry.scales_with and hypercarry.element == 'Pyro')"
# Sara: "'ATK' in hypercarry.scales_with or ('CV' in hypercarry.scales_with and hypercarry.element == 'Electro')"
# Furina: "'DMG' in hypercarry.scales_with"
# Kazuha: "'EM' in hypercarry.scales_with or hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo')"
# Chevruese: "set(stats['elements_in_party']) = set(('Pyro', 'Electro'))"
# Zhongli: "True"

# Relevancy list: elemental resonances
# Anemo: "False"
# Pyro: "'ATK' in hypercarry.scales_with"
# Hydro: "'HP' in hypercarry.scales_with or 'Chevreuse' in buffer_names"
# Electro: "False"
# Cryo: "hypercarry.element in 'Physical', 'Cryo'"
# Geo: "'DMG' in hypercarry.scales_with or hypercarry.element == 'Geo'"
# Dendro: "'EM' in hypercarry.scales_with or 'Kaedehara Kazuha' in buffer_names"

class char_buffer(char_placeholder):
    def __init__(self, name = 'None', element = 'Physical'):
        super(char_buffer, self).__init__(name, element)
        # Abstract party member that provides elemental resonance, but no direct buffs

    def buff(self, stats, hypercarry, weapon):
        return

class widget_buffer(widget_char_placeholder): # Placeholder buffer widget
    def __init__(self):
        super(widget_buffer, self).__init__(char_buffer)

class widget_resonator(widget_char_placeholder):
    def __init__(self):
        super(widget_resonator, self).__init__(char_buffer)
        self.character.name = 'Resonator: Physical'
        #
        label_element = QtWidgets.QLabel('Element:', self)
        self.box_element = make_combo_box(self, ('Physical', 'Anemo', 'Pyro', 'Hydro', 'Electro', 'Cryo', 'Geo', 'Dendro'), self.character.element, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_element, self.box_element): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.element = self.box_element.currentText()
        self.character.name = 'Resonator: ' + self.character.element
        
class char_vv(char_buffer):
    def __init__(self):
        super(char_vv, self).__init__('VV provider', 'Anemo')
        # Somebody doing a VV who is not Kazuha
        # Faruzan can technically hold VV, but she only buffs Anemo damage which can't be VVed so she is a separate case entirely
        
    def buff(self, stats, hypercarry, weapon):
        if hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo'): stats['shred_vv'] = 40
    
class widget_vv(widget_char_placeholder):
    def __init__(self):
        super(widget_vv, self).__init__(char_vv)
        #
        label_vv = QtWidgets.QLabel('Providing elemental RES shred through 4p Viridescent Venerer swirls', self)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_vv,): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))

class char_bennett(char_buffer):
    def __init__(self):
        super(char_bennett, self).__init__('Bennett', 'Pyro')
        # Default Bennett is assumed to have Base ATK of 750 (approx. lvl 90 Sapwood 90), talent level of 8, C0, and has 4p Noblesse Obligue
        self.base_attack = 750
        self.talent = 8
        self.constellation = 0
        self.no4p = True
    
    def buff_value_atk(self):
        return self.base_attack*((56, 60.2, 64.4, 70, 74.2, 78.4, 84, 89.6, 95.2, 100.8, 106.4, 112, 119, 126)[self.talent-1] + (20 if self.constellation >= 1 else 0))/100
    
    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + self.buff_value_atk() + (0.2*hypercarry.base_atk(weapon) if self.no4p else 0)
        stats['DMG'] = stats['DMG'] + (15 if self.constellation == 6 and hypercarry.element == 'Pyro' else 0)

class widget_bennett(widget_char_placeholder): # Widget meant to be put in the stacked widget for buffers in the main window
    def __init__(self):
        super(widget_bennett, self).__init__(char_bennett)
        #
        label_base_atk = QtWidgets.QLabel('Base ATK:', self)
        self.box_base_atk = make_spin_box(self, 0, 9999, self.character.base_attack, self.update_char)
        label_talent = QtWidgets.QLabel('Talent level:', self)
        self.box_talent = make_spin_box(self, 1, 13, self.character.talent, self.update_char)
        label_con = QtWidgets.QLabel('Constellation:', self)
        self.box_con = make_spin_box(self, 0, 6, self.character.constellation, self.update_char)
        self.box_no4p = make_check_box(self, "Holds 4p Noblesse Obligue", self.character.no4p, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_base_atk, self.box_base_atk, label_talent, self.box_talent, label_con, self.box_con, self.box_no4p): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.base_attack = self.box_base_atk.value()
        self.character.talent = self.box_talent.value()
        self.character.constellation = self.box_con.value()
        self.character.no4p = self.box_no4p.isChecked()
        
class char_sara(char_buffer):
    def __init__(self):
        super(char_sara, self).__init__('Sara', 'Electro')
        # Default Sara is assumed to have Base ATK of 750 (approx. lvl 90 565-bow 90), talent level of 8 and no C6
        self.base_attack = 750
        self.talent = 8
        self.c6 = False
        
    def buff_value_atk(self): # CDMG buff is added separately, that's why we need carry's crit stats
        return self.base_attack*((43, 46, 49, 54, 57, 60, 64.4, 69, 73.0, 77.3, 81.6, 85.9, 91.2, 97)[self.talent-1])/100

    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + self.buff_value_atk()
        stats['CV'] = stats['CV'] + (60 if self.c6 == True and hypercarry.element == 'Electro' else 0)

class widget_sara(widget_char_placeholder):
    def __init__(self):
        super(widget_sara, self).__init__(char_sara)
        #
        label_base_atk = QtWidgets.QLabel('Base ATK:', self)
        self.box_base_atk = make_spin_box(self, 0, 9999, self.character.base_attack, self.update_char)
        label_talent = QtWidgets.QLabel('Talent level:', self)
        self.box_talent = make_spin_box(self, 1, 13, self.character.talent, self.update_char)
        self.box_is_c6 = make_check_box(self, "Is C6", self.character.c6, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_base_atk, self.box_base_atk, label_talent, self.box_talent, self.box_is_c6): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.base_attack = self.box_base_atk.value()
        self.character.talent = self.box_talent.value()
        self.character.c6 = self.box_is_c6.isChecked()
        
class char_furina(char_buffer):
    def __init__(self):
        super(char_furina, self).__init__('Furina', 'Hydro')
        # Default Furina is assumed to have average Fanfare charge percent of 75, talent level of 8 and no C1
        self.avg_perc = 75
        self.talent = 8
        self.c1 = False
    
    def buff_value_dmg(self): # This is a DMG% buff, not ATK
        return (0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19, 0.21, 0.23, 0.25, 0.27, 0.29, 0.31)[self.talent-1]*((150+2.5*self.avg_perc) if self.c1 == True else (3*self.avg_perc))
       
    def buff(self, stats, hypercarry, weapon):
        stats['DMG'] = stats['DMG'] + self.buff_value_dmg()

class widget_furina(widget_char_placeholder):
    def __init__(self):
        super(widget_furina, self).__init__(char_furina)
        #
        label_avg_perc = QtWidgets.QLabel('Avg. charge %:', self)
        self.box_avg_perc = make_double_spin_box(self, 0, 100, self.character.avg_perc, self.update_char)
        label_talent = QtWidgets.QLabel('Talent level:', self)
        self.box_talent = make_spin_box(self, 1, 13, self.character.talent, self.update_char)
        self.box_is_c1 = make_check_box(self, "Is C1", self.character.c1, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_avg_perc, self.box_avg_perc, label_talent, self.box_talent, self.box_is_c1): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.avg_perc = self.box_avg_perc.value()
        self.character.talent = self.box_talent.value()
        self.character.c1 = self.box_is_c1.isChecked()
        
class char_kazuha(char_buffer):
    def __init__(self):
        super(char_kazuha, self).__init__('Kaedehara Kazuha', 'Anemo')
        # Default Kazuha is expected to have 800 EM and no C2
        self.em = 800        
        self.c2 = False
        self.vv4p = True
    
    def buff_value_dmg(self, stats):
        return 0.04*(self.em + (200 if self.c2 else 0) + (75 if stats['elements_in_party'].count('Dendro') >= 2 else 0))

    def buff(self, stats, hypercarry, weapon):
        if self.c2: stats['EM'] = stats['EM'] + 200
        if hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo'):
            stats['DMG'] = stats['DMG'] + self.buff_value_dmg(stats)
            if self.vv4p: stats['shred_vv'] = 40
    
class widget_kazuha(widget_char_placeholder):
    def __init__(self):
        super(widget_kazuha, self).__init__(char_kazuha)
        #
        label_em = QtWidgets.QLabel('Elemental mastery:', self)
        self.box_em = make_spin_box(self, 0, 1999, self.character.em, self.update_char)
        self.box_is_c2 = make_check_box(self, "Is C2", self.character.c2, self.update_char)
        self.box_vv4p = make_check_box(self, "Holds 4p Viridescent Venerer", self.character.vv4p, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_em, self.box_em, self.box_is_c2, self.box_vv4p): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.em = self.box_em.value()
        self.character.is_c2 = self.box_is_c2.isChecked()
        self.character.vv4p = self.box_vv4p.isChecked()

class char_chevreuse(char_buffer):
    def __init__(self):
        super(char_chevreuse, self).__init__('Chevreuse', 'Pyro')
        # Default Chevreuse is assumed to have 35k hp, and have 0 C6 stacks active
        self.hp = 35000
        self.c6_stacks = 0
        
    def buff_value_dmg(self):
        return 0.001*self.hp + 20*self.c6_stacks
    
    def buff(self, stats, hypercarry, weapon):
        if set(stats['elements_in_party']) == set(('Pyro', 'Electro')):
            stats['DMG'] = stats['DMG'] + self.buff_value_dmg() # !! UNFINISHED !! Chevreuse instance cannot know if there is a Hydro resonance
            stats['shred_chevreuse'] = 40

class widget_chevreuse(widget_char_placeholder):
    def __init__(self):
        super(widget_chevreuse, self).__init__(char_chevreuse)
        #
        label_hp = QtWidgets.QLabel('HP:', self)
        self.box_hp = make_spin_box(self, 1, 40000, self.character.hp, self.update_char)
        label_c6_stacks = QtWidgets.QLabel('C6 stacks:', self)
        self.box_c6_stacks = make_spin_box(self, 0, 3, self.character.c6_stacks, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_hp, self.box_hp, label_c6_stacks, self.box_c6_stacks): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.hp = self.box_hp.value()
        self.character.c6_stacks = self.box_c6_stacks.value()

class char_zhongli(char_buffer):
    def __init__(self):
        super(char_zhongli, self).__init__('Zhongli', 'Geo')
        # Default Zhongli provides his shield and holds 4p Tenacity of the Millelith
        self.tom4p = True
  
    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + (0.2*hypercarry.base_atk(weapon) if self.tom4p else 0)
        stats['shred_zhongli'] = 20
    
class widget_zhongli(widget_char_placeholder):
    def __init__(self):
        super(widget_zhongli, self).__init__(char_zhongli)
        #
        self.box_tom4p = make_check_box(self, "Holds 4p Tenacity of the Millelith", self.character.tom4p, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (self.box_tom4p,): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.tom4p = self.box_tom4p.isChecked()

In [8]:
class widget_genshin_buff_opt_main(QtWidgets.QWidget):
    def __init__(self):
        super(widget_genshin_buff_opt_main, self).__init__()
        # Alternative, no-preload way to pick enabled hypercarries and buffers - by names
        self.hc_types = {
            'Basic hypercarry': widget_hypercarry,
            'Kamisato Ayaka': widget_ayaka,
            'Hu Tao': widget_hutao,
            'Raiden Shogun': widget_ei,
            'Diluc': widget_diluc,
            'Abstract hypercarry': widget_hypercarry_custom
        }
        self.buffer_types = {
            'None': widget_char_placeholder,
            'Bennett': widget_bennett,
            'Kujou Sara': widget_sara,
            'Furina': widget_furina,
            'Kaedehara Kazuha': widget_kazuha,
            'Chevreuse': widget_chevreuse,
            'Zhongli': widget_zhongli,
            'VV provider': widget_vv,
            'Resonance provider': widget_resonator
        }
        self.hc_widgets = dict()
        for item in self.hc_types.keys(): self.hc_widgets[item] = widget_char_placeholder()
        # These arcane runes mean 'get first key and value from a dictionary' because even ordered since 3.7, they don't support indexing
        self.hc_widgets[tuple(self.hc_types.keys())[0]] = (tuple(self.hc_types.values())[0])()
        self.buffer_widgets = dict()
        for item in self.buffer_types.keys(): self.buffer_widgets[item] = widget_char_placeholder()
        buffer_names = tuple(self.buffer_types.keys())
        self.placeholder_buffer_widgets = (widget_buffer(), widget_buffer(), widget_buffer()) # These are for the fields, not for preloads
        self.enemy_res = 10
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        # Hypercarry
        layout_hc_s = QtWidgets.QHBoxLayout()
        label_hc = QtWidgets.QLabel('Hypercarry:', self)
        self.box_hc = make_combo_box(self, tuple(self.hc_widgets.keys()), tuple(self.hc_types.keys())[0], self.switch_hypercarry)
        label_res = QtWidgets.QLabel('Enemy elemental resistance:', self)
        self.box_res = make_spin_box(self, -100, 999, self.enemy_res, self.update_enemy_res)
        for item in (label_hc, self.box_hc, label_res, self.box_res): layout_hc_s.addWidget(item)
        layout_hc_s.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout_hc = QtWidgets.QStackedLayout()
        for item in self.hc_widgets.values(): self.layout_hc.addWidget(item) # Populate with the default and placeholder widgets to avoid preloading hypercarries
        frame_hc = make_basic_frame(self.layout_hc)
        # Buffers
        self.layouts_buffers = (QtWidgets.QVBoxLayout(), QtWidgets.QVBoxLayout(), QtWidgets.QVBoxLayout()) # I need to keep those named to be able to reference them to switch out buffers
        self.boxes_buffers = (QtWidgets.QComboBox(self), QtWidgets.QComboBox(self), QtWidgets.QComboBox(self))
        for i in range(3):
            layout_buffer = QtWidgets.QHBoxLayout()
            label_buffer = QtWidgets.QLabel('Buffer ' + str(i+1) + ':', self)
            self.boxes_buffers[i].addItems(buffer_names)
            self.boxes_buffers[i].activated.connect(self.switch_buffers)
            layout_buffer.addWidget(label_buffer)
            layout_buffer.addWidget(self.boxes_buffers[i])
            layout_buffer.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
            self.layouts_buffers[i].addLayout(layout_buffer)
            self.layouts_buffers[i].addWidget(self.placeholder_buffer_widgets[i])
            self.layouts_buffers[i].addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
            del label_buffer
            del layout_buffer
        # Buttons
        layout_buttons = QtWidgets.QVBoxLayout(self)
        button_calculate = QtWidgets.QPushButton('Calculate damage output for the current setup', self)
        button_calculate.clicked.connect(self.calculate_setup)
        self.label_calc_result = QtWidgets.QLabel('Current configuration output is:', self)
        self.label_stats_result = QtWidgets.QLabel('Effective stats:', self)
        layout_buttons.addWidget(button_calculate)
        layout_buttons.addWidget(self.label_calc_result)
        layout_buttons.addWidget(self.label_stats_result)
        # Optimizers
        layout_opt_s = QtWidgets.QHBoxLayout()
        self.button_opt_build = QtWidgets.QPushButton('Build', self)
        self.button_opt_build.clicked.connect(self.switch_optimizer)
        self.button_opt_buffers = QtWidgets.QPushButton('Buffers', self)
        self.button_opt_buffers.clicked.connect(self.switch_optimizer)
        layout_opt_s.addWidget(self.button_opt_build)
        layout_opt_s.addWidget(self.button_opt_buffers)
        layout_opt_s.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        widget_opt_build = widget_optimize_build(self.calculate_output, self.layout_hc, self.layouts_buffers)
        widget_opt_buffers = widget_optimize_buffers(self.calculate_output, self.layout_hc, self.buffer_types, self.get_buffer_widget)
        self.layout_optimizers = QtWidgets.QStackedLayout()
        for item in (widget_opt_build, widget_opt_buffers): self.layout_optimizers.addWidget(item)
        frame_opt = make_basic_frame(self.layout_optimizers)
        #
        for item in (layout_hc_s, frame_hc, self.layouts_buffers[0], self.layouts_buffers[1], self.layouts_buffers[2], layout_buttons, layout_opt_s, frame_opt):
            if type(item) in (QtWidgets.QHBoxLayout, QtWidgets.QVBoxLayout, QtWidgets.QGridLayout, QtWidgets.QStackedLayout): layout_main.addLayout(item)
            else: layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    # Note: 'hypercarry' and 'buffer' may be used interchangeably as name strings or character objects in different functions, pay attention
        
    def update_enemy_res(self):
        self.enemy_res = float(self.box_res.text())
    
    def get_hc_widget(self, hc_name):
        # Check if this hypercarry widget has already been made, make it and insert it into the hypercarry QStackedLayout if not, return it
        if self.hc_widgets[hc_name].character.name == 'None':
            old_w = self.hc_widgets[hc_name]
            self.hc_widgets[hc_name] = self.hc_types[hc_name]()
            self.layout_hc.replaceWidget(old_w, self.hc_widgets[hc_name])
            old_w.deleteLater()
            old_w = None
        return self.hc_widgets[hc_name]

    def switch_hypercarry(self):
        hc_name = self.box_hc.currentText()
        if hc_name != self.layout_hc.currentWidget().character.name: # Checking that the user actually picked someone new
            self.get_hc_widget(hc_name)
            self.layout_hc.setCurrentIndex(tuple(self.hc_widgets.keys()).index(hc_name))
    
    def get_buffer_widget(self, buffer_name):
        # Check if this buffer widget has already been made, make it if not, return it if yes
        if self.buffer_widgets[buffer_name].character.name == 'None':
            self.buffer_widgets[buffer_name] = self.buffer_types[buffer_name]()
        return self.buffer_widgets[buffer_name]
        
    def switch_buffers(self):
        num_changed = self.boxes_buffers.index(self.sender()) # Looks stupid, but apparently it's not possible to send an argument with a connect()
        buffer_names = (self.boxes_buffers[0].currentText(), self.boxes_buffers[1].currentText(), self.boxes_buffers[2].currentText())
        # I can't figure out a way to selectively disable and reenable elements in a QComboBox at my current Qt level, so here's a workaround
        # You selected the same buffer twice? Change nothing, reset selection in the box you just tried to change to what it was
        if buffer_names.count(buffer_names[num_changed]) > 1 and buffer_names[num_changed] not in ('None', 'Resonance provider'):
            previous_buffer_name = (self.layouts_buffers[num_changed]).itemAt(1).widget().character.name
            (self.boxes_buffers[num_changed]).setCurrentText(previous_buffer_name)
        elif buffer_names[num_changed] != (self.layouts_buffers[num_changed]).itemAt(1).widget().character.name:
            # Now, I need to replace whichever buffer widget happens to currently be in the slot with the new one - if the new one is a new one
            old_w = (self.layouts_buffers[num_changed]).itemAt(1).widget()
            if buffer_names[num_changed] == 'None':
                new_w = self.placeholder_buffer_widgets[num_changed] # Getting the correct buffer 'None' placeholder widget for this buffer slot
            elif buffer_names[num_changed] == 'Resonance provider': # Resonator character.name depend on their element, but here it is just the box text
                new_w = widget_resonator() # Make a new one since we don't need to keep track of them at all
            else:
                new_w = self.get_buffer_widget(buffer_names[num_changed])
            old_w.hide()
            (self.layouts_buffers[num_changed]).replaceWidget(old_w, new_w) # Resonator widget should be lost now. I can delete it explicitly too I think, but there's no need
            new_w.show()
            
    # Note: resonator widgets and buffer slot placeholder widgets are processed separately even though their functionality is the same - we don't need to keep track of them beyond there being enough
    # It would be slicker to handle both like the resonator, but I'll leave it as-is as a showcase

    def switch_optimizer(self):
        opt_index = (self.button_opt_build, self.button_opt_buffers).index(self.sender())
        self.layout_optimizers.setCurrentIndex(opt_index)

    def get_current_buffers(self):
        # Return a tuple of currently selected widget_buffer.character's
        # These arcane runes send the character objects placed within the widgets put in the frames by buffer selectors to the hypercarry object. If there is no buffer, the placeholder sends a 'None'
        return (self.layouts_buffers[0].itemAt(1).widget().character, self.layouts_buffers[1].itemAt(1).widget().character, self.layouts_buffers[2].itemAt(1).widget().character)
    
    def res_mult(self, res):
        # Using the formulas from the wiki
        if res > 75:
            return 1/(4*res/100 + 1)
        elif res > 0:
            return 1 - res/100
        else:
            return 1 - res/200
        
    def get_elements_in_party(self, hypercarry, buffers):
        elements = [hypercarry.element]
        for buffer in buffers:
            if buffer.element != 'None': elements.append(buffer.element)
        return elements        

    def calculate_output(self, hypercarry = None, weapon = None, artifacts = None, buffers = None):
        if weapon == None:
            weapon = hypercarry.weapon
        stats = hypercarry.calculate_stats(weapon = weapon, artifacts = artifacts)
        enemy_res = self.enemy_res # There are a lot of characters that can offer small RES shred, but the main are VV, Chevreuse and Zhongli
        stats['elements_in_party'] = self.get_elements_in_party(hypercarry, buffers)
        # Elemental resonances - the ones that count are Pyro, Hydro, Cryo, Geo and Dendro
        if stats['elements_in_party'].count('Pyro') >= 2:
            stats['ATK'] = stats['ATK'] + 0.25*hypercarry.base_atk(weapon)
        if stats['elements_in_party'].count('Hydro') >= 2:
            stats['HP'] = stats['HP'] + 0.25*hypercarry.base_stats['HP'] # This should also increase Chevreuse's HP, but we don't even know what level she is
        if stats['elements_in_party'].count('Cryo') >= 2:
            stats['CV'] = stats['CV'] + 30 # Not really true for Melt, but whatever
        if stats['elements_in_party'].count('Geo') >= 2:
            stats['DMG'] = stats['DMG'] + 15
            if hypercarry.element == 'Geo': enemy_res = enemy_res - 20
        if stats['elements_in_party'].count('Dendro') >= 2:
            stats['EM'] = stats['EM'] + 75 # Average since we don't know the reactions
        # Buffs
        stats = hypercarry.apply_self_buffs(stats)
        stats = weapon.apply_buffs(stats)
        for buffer in buffers: buffer.buff(stats, hypercarry, weapon)
        for shred in ['shred_vv', 'shred_chevreuse', 'shred_zhongli']:
            if shred in stats: enemy_res = enemy_res - stats[shred]
        # Conversions
        stats = hypercarry.apply_conversions(stats)
        stats = weapon.apply_conversions(stats)
        stats['CR'] = min(stats['CV']/4, 100)
        stats['CD'] = stats['CV'] - 2*stats['CR']
        return stats['ATK']*(1 + stats['DMG']/100)*(1 + (stats['CR']/100)*(stats['CD']/100))*self.res_mult(enemy_res), stats
    
    def format_stats(self, stats):
        stats_formatted = ''
        shred_sum = 0
        for key, value in stats.items():
            if key in ('HP', 'ATK', 'DEF', 'EM'): stats_formatted = stats_formatted + key + ': ' + "{:.0f}".format(value) + ' | '
            if key in ('ER', 'DMG', 'CR', 'CD'): stats_formatted = stats_formatted + key + ': ' + "{:.1f}".format(value) + '% | '
            if 'shred' in key: shred_sum = shred_sum + stats[key]
        stats_formatted = stats_formatted + 'RES shred: ' + "{:.1f}".format(shred_sum)
        return stats_formatted
            
    def calculate_setup(self):
        output, stats = self.calculate_output(self.layout_hc.currentWidget().character, buffers = self.get_current_buffers())
        self.label_calc_result.setText('Current configuration output is: ' + "{:.1f}".format(output))
        stats_formatted = self.format_stats(stats)
        self.label_stats_result.setText('Effective stats: ' + stats_formatted)

In [9]:
class widget_optimize_build(QtWidgets.QWidget):
    def __init__(self, calculator, hc_container, buffers_container):
        super(widget_optimize_build, self).__init__()
        self.calculate_output = calculator
        self.hc_container = hc_container # Giving the optimizer containers from the main widget so they can get the current thing from them
        self.buffers_container = buffers_container
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_checks = QtWidgets.QHBoxLayout()
        self.box_opt_weapon = make_check_box(self, 'Weapon', True)
        self.box_opt_sands = make_check_box(self, 'Sands', True)
        self.box_opt_goblet = make_check_box(self, 'Goblet', True)
        self.box_opt_circlet = make_check_box(self, 'Circlet', True)
        for item in (self.box_opt_weapon, self.box_opt_sands, self.box_opt_goblet, self.box_opt_circlet): layout_checks.addWidget(item)
        layout_checks.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        layout_main.addLayout(layout_checks)
        button_optimize = QtWidgets.QPushButton('Optimize', self)
        button_optimize.clicked.connect(self.optimize_build)
        self.label_result = QtWidgets.QLabel('Best configuration is:', self)
        for item in (button_optimize, self.label_result): layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    def optimize_build(self):
        # Calculate output for weapon and artifact (main stats only) setup variations with the current buffer configuration
        hypercarry = self.hc_container.currentWidget().character
        buffers = (self.buffers_container[0].itemAt(1).widget().character, self.buffers_container[1].itemAt(1).widget().character, self.buffers_container[2].itemAt(1).widget().character)
        #
        if self.box_opt_weapon.isChecked(): weapon_types = list(hypercarry.weapon_types.keys())
        else: weapon_types = (hypercarry.weapon.name,)
        if self.box_opt_sands.isChecked(): sands_types = hypercarry.sands_types
        else: sands_types = (hypercarry.artifacts[0],)
        if self.box_opt_goblet.isChecked(): goblet_types = hypercarry.goblet_types
        else: goblet_types = (hypercarry.artifacts[1],)
        if self.box_opt_circlet.isChecked(): circlet_types = hypercarry.circlet_types
        else: circlet_types = (hypercarry.artifacts[2],)
        outputs = list()
        for weapon, sands, goblet, circlet in product(weapon_types, sands_types, goblet_types, circlet_types):
            output, stats = self.calculate_output(hypercarry, hypercarry.weapon_types[weapon], (sands, goblet, circlet), buffers)
            outputs.append([output, (weapon, sands, goblet, circlet)])
        best = max(outputs) # Should automatially give the pair with highest output number
        self.label_result.setText('Best configuration is: ' + str(best[1]) + " with the output of " + "{:.1f}".format(best[0]))
    
class widget_optimize_buffers(QtWidgets.QWidget):
    def __init__(self, calculator, hc_container, buffer_types, get_buffer_widget):
        super(widget_optimize_buffers, self).__init__()
        self.calculate_output = calculator
        self.hc_container = hc_container
        self.buffer_types = buffer_types
        self.get_buffer_widget = get_buffer_widget
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_options = QtWidgets.QHBoxLayout()
        layout_num_buffers = QtWidgets.QVBoxLayout()
        label_num_buffers = QtWidgets.QLabel('Number of buffers:', self)
        self.box_num_buffers = make_spin_box(self, 1, 3, 1) # Doesn't need to be connected as we are only looking at it with methods
        layout_num_buffers.addWidget(label_num_buffers)
        layout_num_buffers.addWidget(self.box_num_buffers)
        layout_num_buffers.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        layout_available = QtWidgets.QVBoxLayout()
        container_available = QtWidgets.QWidget()
        container_available.setLayout(layout_available)
        self.boxes_available = dict()
        buffer_names = list(self.buffer_types.keys())
        buffer_names.pop()
        buffer_names.pop(0)
        for name in buffer_names:
            self.boxes_available[name] = make_check_box(self, name, True)
            layout_available.addWidget(self.boxes_available[name])
        scroll_available = QtWidgets.QScrollArea()
        scroll_available.setWidget(container_available)
        scroll_available.setWidgetResizable(True)
        scroll_available.setFixedHeight(80)
        layout_options.addLayout(layout_num_buffers)
        layout_options.addWidget(scroll_available)
        button_optimize = QtWidgets.QPushButton('Optimize', self)
        button_optimize.clicked.connect(self.optimize_buffers)
        self.label_result = QtWidgets.QLabel('Best configuration is:', self)
        layout_main.addLayout(layout_options)
        for item in (button_optimize, self.label_result): layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))

    def optimize_buffers(self):
        # Calculate output for buffers setup variations and update the result label with the best
        # Ideally, the list of buffers should contain resonators relevant the HC's scalings, then buffers relevant to HC's scalings
        hypercarry = self.hc_container.currentWidget().character
        #
        buffer_names = list()
        for key, value in self.boxes_available.items():
            if value.isChecked(): buffer_names.append(key) # !! UNFINISHED !! Add relevancy here
        outputs = list()
        for buffer_combo in combinations(buffer_names, self.box_num_buffers.value()):
            buffers = list()
            for name in buffer_combo:
                buffers.append(self.get_buffer_widget(name).character) # Combine by names, send character objects
            output, stats = self.calculate_output(hypercarry, buffers = buffers)
            outputs.append([output, buffer_combo])
        if len(outputs) == 0: self.label_result.setText('No combinations available')            
        else:
            best = max(outputs)
            self.label_result.setText('Best buffers are: ' + str(best[1]) + " with the output of " + "{:.1f}".format(best[0]))

In [10]:
class this_main_window(QtWidgets.QMainWindow):
    def __init__(self):
        super(this_main_window, self).__init__()
        main_widget = widget_genshin_buff_opt_main()
        self.setCentralWidget(main_widget)
        self.setWindowTitle("GI Hypercarry Optimizer")
        self.setGeometry(50, 100, 750, 600)    
    
if __name__ == '__main__':
    import sys
    # Modified version of the runner code for Jupyter compatibility
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    window = this_main_window()
    window.show()
    app.exec_()
    del app